In [1]:
import os
from PIL import Image
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers, losses

In [2]:
train_dir = "Train"

# Train augmentation

In [3]:
datagen = ImageDataGenerator(validation_split=0.1, rescale=1./255)

train_generator = datagen.flow_from_directory(
    train_dir,
    subset='training',
    target_size=(30,30),
    batch_size=32,
    color_mode='rgb',    
    shuffle=True,
    seed=42,
    class_mode='categorical')

val_generator = datagen.flow_from_directory(
    #flow_from_directory method will identify classes automatically from the folder name.
    #This method is useful when the images are sorted and placed in there respective class/label folders.
    train_dir,
    subset='validation',
    target_size=(30,30),
    batch_size=32,
    color_mode='rgb',    
    shuffle=True,
    seed=42,
    class_mode='categorical')

Found 35289 images belonging to 43 classes.
Found 3920 images belonging to 43 classes.


In [4]:
input_shape = (30,30,3) # img_rows, img_colums, color_channels
num_classes = 43

In [5]:
## Build Model
inputs = layers.Input(shape=input_shape)
# 1st Conv layer 
x = layers.Conv2D(32, kernel_size = (3, 3), activation = 'relu', padding = 'same')(inputs)
x = layers.Conv2D(32, kernel_size = (3, 3), activation = 'relu', padding = 'same')(x)
x = layers.MaxPool2D(pool_size = (2, 2))(x)
# 2nd Conv layer        
x = layers.Conv2D(64, kernel_size = (3, 3), activation = 'relu', padding = 'same')(x)
x = layers.Conv2D(64, kernel_size = (3, 3), activation = 'relu', padding = 'same')(x)
x = layers.MaxPool2D(pool_size = (2, 2))(x)
# Fully Connected layer        
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = models.Model(inputs=inputs, outputs=outputs)

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 30, 30, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0     

In [7]:
# Compile Model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model

In [8]:
print(train_generator.n//train_generator.batch_size)
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VAL =val_generator.n//val_generator.batch_size
num_epochs = 10

1102


In [9]:
# Train Model
history = model.fit(train_generator,steps_per_epoch=STEP_SIZE_TRAIN,epochs=num_epochs, validation_data=val_generator, validation_steps=STEP_SIZE_VAL) #, callbacks=[checkpoint])

Epoch 1/10
1102/1102 [==============================] - 138s 124ms/step - loss: 0.7372 - accuracy: 0.7921 - val_loss: 0.2965 - val_accuracy: 0.9247
Epoch 2/10
1102/1102 [==============================] - 110s 100ms/step - loss: 0.0919 - accuracy: 0.9735 - val_loss: 0.3011 - val_accuracy: 0.9306
Epoch 3/10
1102/1102 [==============================] - 112s 102ms/step - loss: 0.0547 - accuracy: 0.9840 - val_loss: 0.2451 - val_accuracy: 0.9385oss: 0.054
Epoch 4/10
1102/1102 [==============================] - 113s 102ms/step - loss: 0.0394 - accuracy: 0.9881 - val_loss: 0.2992 - val_accuracy: 0.9339
Epoch 5/10
1102/1102 [==============================] - 115s 104ms/step - loss: 0.0392 - accuracy: 0.9883 - val_loss: 0.2630 - val_accuracy: 0.9329
Epoch 6/10
1102/1102 [==============================] - 117s 106ms/step - loss: 0.0287 - accuracy: 0.9914 - val_loss: 0.2152 - val_accuracy: 0.9485
Epoch 7/10
1102/1102 [==============================] - 114s 104ms/step - loss: 0.0255 - accuracy: 0.9

In [10]:
score = model.evaluate(val_generator, steps=STEP_SIZE_VAL)
print('VAL loss :', score[0])
print('VAL accuracy:', score[1])

122/122 [==============================] - 3s 25ms/step - loss: 0.2772 - accuracy: 0.9541 2s - loss: 0.3418 - accuracy: 0.95 - E
VAL loss : 0.277172714471817
VAL accuracy: 0.954149603843689


# Test Model

In [11]:
import pandas as pd
df = pd.read_csv('Test.csv')
df.head()

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path
0,53,54,6,5,48,49,16,Test/00000.png
1,42,45,5,5,36,40,1,Test/00001.png
2,48,52,6,6,43,47,38,Test/00002.png
3,27,29,5,5,22,24,33,Test/00003.png
4,60,57,5,5,55,52,11,Test/00004.png


In [12]:
df.dtypes

Width       int64
Height      int64
Roi.X1      int64
Roi.Y1      int64
Roi.X2      int64
Roi.Y2      int64
ClassId     int64
Path       object
dtype: object

In [8]:
import os
!mkdir -p Test1

from shutil import copyfile
src_dir  = ""
test_dir = "Test1/"
for i in range(len(df)):
    if not os.path.exists(test_dir+str(df.iloc[i].ClassId)):
        os.makedirs(test_dir+str(df.iloc[i].ClassId))
    copyfile(src_dir+df.iloc[i].Path, test_dir+str(df.iloc[i].ClassId)+'/'+df.iloc[i].Path[5:])

A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file Test1 already exists.
Error occurred while processing: Test1.


In [13]:
test_dir = "Test1/"
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(30,30),
    batch_size=32,
    color_mode='rgb',    
    shuffle=True,
    seed=42,
    class_mode='categorical')

Found 12630 images belonging to 43 classes.


In [14]:
STEP_SIZE_TEST =test_generator.n//test_generator.batch_size

score = model.evaluate_generator(test_generator, steps=STEP_SIZE_TEST)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\keras\engine\training.py:2006: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


Test loss: 0.20715554058551788
Test accuracy: 0.9642290472984314


In [15]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

pred

  1/394 [..............................] - ETA: 1:01

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\keras\engine\training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


394/394 [==============================] - 10s 25ms/step


array([[1.65177538e-04, 1.47913738e-06, 1.63709883e-05, ...,
        3.10875148e-05, 9.97283578e-01, 2.01800038e-04],
       [3.01416674e-14, 6.45793965e-12, 1.30745526e-13, ...,
        2.45298616e-12, 1.05539154e-10, 7.54298313e-09],
       [6.12126712e-12, 6.75872943e-12, 4.41895005e-20, ...,
        1.62719859e-12, 3.35056310e-13, 1.50806658e-21],
       ...,
       [1.36130780e-07, 5.86239279e-10, 2.68002714e-06, ...,
        1.59703839e-09, 2.92975999e-09, 6.58572731e-07],
       [7.92060848e-14, 5.40112953e-21, 1.52480201e-13, ...,
        1.05131148e-13, 1.00000000e+00, 2.96346553e-18],
       [4.52802183e-18, 1.14425878e-10, 5.09667083e-16, ...,
        1.00378039e-11, 9.27303041e-15, 2.02558975e-18]], dtype=float32)

In [16]:
predicted_class_indices=np.argmax(pred,axis=1)
predicted_class_indices

array([41,  5, 34, ..., 11, 41, 38], dtype=int64)

In [17]:
np.amax(predicted_class_indices)

42

In [18]:
labels = (train_generator.class_indices)
labels

{'0': 0,
 '1': 1,
 '10': 2,
 '11': 3,
 '12': 4,
 '13': 5,
 '14': 6,
 '15': 7,
 '16': 8,
 '17': 9,
 '18': 10,
 '19': 11,
 '2': 12,
 '20': 13,
 '21': 14,
 '22': 15,
 '23': 16,
 '24': 17,
 '25': 18,
 '26': 19,
 '27': 20,
 '28': 21,
 '29': 22,
 '3': 23,
 '30': 24,
 '31': 25,
 '32': 26,
 '33': 27,
 '34': 28,
 '35': 29,
 '36': 30,
 '37': 31,
 '38': 32,
 '39': 33,
 '4': 34,
 '40': 35,
 '41': 36,
 '42': 37,
 '5': 38,
 '6': 39,
 '7': 40,
 '8': 41,
 '9': 42}

In [19]:
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
print(labels)
print(predictions)

{0: '0', 1: '1', 2: '10', 3: '11', 4: '12', 5: '13', 6: '14', 7: '15', 8: '16', 9: '17', 10: '18', 11: '19', 12: '2', 13: '20', 14: '21', 15: '22', 16: '23', 17: '24', 18: '25', 19: '26', 20: '27', 21: '28', 22: '29', 23: '3', 24: '30', 25: '31', 26: '32', 27: '33', 28: '34', 29: '35', 30: '36', 31: '37', 32: '38', 33: '39', 34: '4', 35: '40', 36: '41', 37: '42', 38: '5', 39: '6', 40: '7', 41: '8', 42: '9'}
['8', '13', '4', '25', '5', '33', '10', '15', '1', '14', '23', '1', '4', '8', '2', '38', '2', '9', '26', '26', '8', '35', '38', '7', '7', '15', '17', '13', '38', '14', '5', '13', '18', '3', '13', '8', '8', '10', '8', '10', '18', '25', '4', '16', '18', '5', '14', '2', '8', '17', '1', '4', '12', '11', '12', '35', '14', '8', '17', '6', '35', '25', '25', '4', '5', '2', '38', '14', '11', '38', '3', '9', '23', '2', '18', '25', '5', '5', '7', '25', '28', '36', '24', '2', '4', '11', '11', '5', '34', '25', '9', '11', '10', '11', '17', '34', '33', '2', '4', '10', '1', '6', '1', '12', '38', '4

In [20]:
predictions[517]

'1'

# Testing for new images

In [29]:
test_dir = "Test2/"
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(30,30),
    batch_size=4,
    color_mode='rgb',    
    shuffle=True,
    seed=42,
    class_mode='categorical')

STEP_SIZE_TEST =test_generator.n//test_generator.batch_size

"""score = model.evaluate_generator(test_generator, steps=STEP_SIZE_TEST)
print('Test loss:', score[0])
print('Test accuracy:', score[1])"""

test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

print(pred)

predicted_class_indices=np.argmax(pred,axis=1)
print(predicted_class_indices)

labels = (train_generator.class_indices)

labels = dict((v,k) for k,v in labels.items())


import pandas as pd
df=pd.read_csv('mapping.csv')  
v=0
for key, value in labels.items():
    labels[key] = df['STATUS'][v]
    v=v+1
print(labels)


predictions = [labels[k] for k in predicted_class_indices]

print(predictions)




Found 4 images belonging to 1 classes.
1/1 [==============================] - 0s 50ms/step
[[1.19589950e-21 8.77425252e-15 1.81822902e-21 2.41757555e-18
  4.50236055e-20 1.44006983e-17 3.15926680e-17 2.03293434e-15
  2.26527262e-16 2.01067008e-21 7.29618214e-21 1.08936717e-17
  1.01035825e-12 5.87971329e-21 1.01797509e-21 1.19262570e-18
  3.83571177e-18 5.89106188e-25 4.93129616e-19 2.58402481e-23
  1.34577271e-19 1.77823533e-17 4.76507186e-17 1.00000000e+00
  2.36824588e-20 4.96942815e-17 7.54514291e-16 3.55364500e-17
  3.54052339e-15 8.37090484e-18 1.20130057e-19 3.54172849e-21
  8.30808969e-22 2.00149104e-17 1.93663322e-20 8.74666014e-20
  4.93921344e-15 2.57267000e-19 2.28754109e-13 5.74925171e-14
  5.89623860e-23 4.93391830e-24 4.38761007e-11]
 [9.99984503e-01 7.14718840e-10 1.30340424e-16 3.67429630e-21
  6.47881155e-16 1.31306327e-20 1.96900940e-14 4.85288734e-16
  6.32233597e-16 2.12704832e-12 1.46538292e-14 9.51970483e-16
  2.86652676e-21 1.06028747e-15 9.92821462e-20 1.543705

In [26]:
test_generator